In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [3]:
spark = SparkSession.builder.appName('MostObscureHero').getOrCreate()

25/11/19 01:56:39 WARN Utils: Your hostname, turan-H110 resolves to a loopback address: 127.0.1.1; using 192.168.1.9 instead (on interface enp2s0)
25/11/19 01:56:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/19 01:56:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/19 01:56:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/11/19 01:56:41 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
schema = StructType([StructField("id", IntegerType(), True), StructField("name", StringType(), True)])

In [5]:
names = spark.read.schema(schema).option("sep", " ").csv("file:///home/turan/SparkCourse/section_4/Marvel-Names")

In [6]:
lines = spark.read.text("file:///home/turan/SparkCourse/section_4/Marvel-Graph")

In [7]:
preparedHeroDF = lines.withColumn("id", func.split(func.col("value"), " ")[0]) \
    .withColumn("connections", func.size(func.split(func.col("value"), " ")) - 1) \
    .groupBy("id").agg(func.sum("connections").alias("connections"))

In [8]:
mostObscureHeros = preparedHeroDF.sort(func.col("connections").asc())

In [9]:
mostObscureHeros.show()

+----+-----------+
|  id|connections|
+----+-----------+
|4517|          1|
| 577|          1|
|3490|          1|
|3489|          1|
|2139|          1|
|1089|          1|
|1841|          1|
| 467|          1|
|5028|          1|
| 835|          1|
|1408|          1|
|4784|          1|
|4945|          1|
|4602|          1|
|6411|          1|
|3014|          1|
|3298|          1|
|2911|          1|
|2117|          1|
|4364|          2|
+----+-----------+
only showing top 20 rows



In [11]:
filteredMostObscureHeros = mostObscureHeros.filter(func.col("connections") == 1).select("id")

In [12]:
filteredMostObscureHeros.show()

+----+
|  id|
+----+
| 467|
| 577|
|3490|
|3489|
|2139|
|1089|
|1841|
|4517|
|5028|
| 835|
|1408|
|4784|
|4945|
|4602|
|6411|
|3014|
|3298|
|2911|
|2117|
+----+



In [15]:
joinedFilteredMostObscureHeros = filteredMostObscureHeros.join(names, on="id", how = "inner").show()

+----+--------------------+
|  id|                name|
+----+--------------------+
| 467|        BERSERKER II|
| 577|              BLARE/|
|3490|MARVEL BOY II/MARTIN|
|3489|MARVEL BOY/MARTIN BU|
|2139|      GIURESCU, RADU|
|1089|       CLUMSY FOULUP|
|1841|              FENRIS|
|4517|              RANDAK|
|5028|           SHARKSKIN|
| 835|     CALLAHAN, DANNY|
|1408|         DEATHCHARGE|
|4784|                RUNE|
|4945|         SEA LEOPARD|
|4602|         RED WOLF II|
|6411|              ZANTOR|
|3014|JOHNSON, LYNDON BAIN|
|3298|          LUNATIK II|
|2911|                KULL|
|2117|GERVASE, LADY ALYSSA|
+----+--------------------+

